In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


ModuleNotFoundError: No module named 'kagglehub'

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

petfinder_pawpularity_score_path = kagglehub.competition_download('petfinder-pawpularity-score')
awsaf49_vit_keras_dataset_path = kagglehub.dataset_download('awsaf49/vit-keras-dataset')
awsaf49_validators0182_dataset_path = kagglehub.dataset_download('awsaf49/validators0182-dataset')
rishigami_swintransformertf_path = kagglehub.dataset_download('rishigami/swintransformertf')
ludovick_petfindertransformers_path = kagglehub.dataset_download('ludovick/petfindertransformers')
kozodoi_timm_pytorch_image_models_path = kagglehub.dataset_download('kozodoi/timm-pytorch-image-models')

print('Data source import complete.')


100%|██████████| 983M/983M [00:09<00:00, 106MB/s]

Extracting files...


100%|██████████| 54.2k/54.2k [00:00<00:00, 23.9MB/s]

Extracting files...


100%|██████████| 17.2k/17.2k [00:00<00:00, 16.2MB/s]

...

Extracting files...


100%|██████████| 24.7G/24.7G [05:50<00:00, 75.8MB/s]

Extracting files...


100%|██████████| 1.26M/1.26M [00:00<00:00, 92.6MB/s]

Extracting files...
Data source import complete.


In [ ]:
# Shiro

In [9]:
#### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
import torch
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
print(torch.__version__)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2.6.0+cu124


In [7]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [5]:
import timm
SEED = 42
import random
def seed_everything(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)


    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
seed_everything(SEED)

In [10]:
df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../input/petfinder-pawpularity-score/test.csv'

In [8]:
old_folder = "../input/petfinder-pawpularity-score/test"
df_test["path"] = df_test.Id.apply(lambda x: os.path.join(old_folder, x+".jpg"))

NameError: name 'df_test' is not defined

In [11]:
from torch import nn


class PetNet(torch.nn.Module):
    def __init__(self, backbone, dim, num_classes=101, output="classification", one_hot=False):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(PetNet, self).__init__()

        self.type = output
        self.backbone = backbone
        self.one_hot = one_hot
        if one_hot:
            dim = dim + 12
        self.last_layer = nn.Linear(dim, num_classes)
        self.r = torch.arange(num_classes).unsqueeze(0)

    def to_device(self, device):
        self.r = self.r.to(device)
    def forward(self, x, x2=None):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        features = self.backbone(x)
        #print(len(features),features[0].shape, features[1].shape)
        #print(features.shape)
        if type(features) == tuple:
            features = features[0]
        #print(features.shape, x2.shape)
        if self.one_hot:
            features = torch.cat((features, x2), dim=-1)

        #print(features.shape)
        output_proba = self.last_layer(features)

        # if self.type == "regression":
        #     output = nn.functional.softmax(output_proba, dim=1)
        #     #print(output)

        #     output = (self.r * output).sum(1)
        #     return output, output_proba
        #     #print(output.shape)

        #print(output_proba.shape)
        return output_proba

In [ ]:
# evaluation     segment
def inference_fn(model, dataloader, cfg):
    t=tqdm(dataloader, disable=not cfg.verbose)

    y_preds = []
    model.eval()
    device = cfg.device
    with torch.no_grad():
        for i, batch in enumerate(t):

            if len(batch) == 2:
                inputs, indices = batch
                inputs2 = None
            else:
                inputs, inputs2, indices = batch
                inputs2 = inputs2.to(cfg.device, dtype=torch.float)

            inputs = inputs.to(device, dtype=torch.float)

            with autocast(cfg.use_apex_val):
                outputs = model(inputs, x2=inputs2)
                if type(outputs) == tuple:
                    outputs = outputs[0]

            if cfg.mode == "classification":
                y_preds.append( torch.nn.functional.softmax(outputs, dim=1).cpu().detach().numpy())
            else:
                y_preds.append( outputs.cpu().detach().numpy())
    return np.concatenate(y_preds).astype(np.float32)

In [ ]:
import torch
import os
import pandas as pd
import cv2
from torchvision import transforms as pth_transforms
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset, DataLoader
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

class PetDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data,one_hot=False,resize=224):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data["path"].values.tolist()
        self.one_hot = one_hot
        if one_hot: # 12 feature
            self.data2 = data[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
                               'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']].values
        self.transform = pth_transforms.Compose([
                                                 pth_transforms.ToPILImage(),
                                                 pth_transforms.Resize((resize, resize)),
                                                 #pth_transforms.CenterCrop(224),
                                                 pth_transforms.ToTensor(),
                                                 pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                             ])

    def __getitem__(self, idx):
        img = cv2.imread(self.data[idx])
        #img = cv2.resize(img, (224,224))
        img = self.transform(img)
        if self.one_hot:
            return img, self.data2[idx], idx
        return img, idx

    def __len__(self):
        return len(self.data)

In [ ]:
class Config(object):
    def __init__(self):
        self.epochs = 100
        self.lr = 1e-4          # learning rate
        self.wd = 1e-5
        self.accumulation = 1
        self.batch_size=32
        self.use_apex = True
        self.use_apex_val = False
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.num_classes = 101
        self.model_name = "cait_m36_384" #"cait_m36_384"#'swin_large_patch4_window12_384_in22k'
        self.kfold=5
        self.resize = 384
        self.one_hot= True
        self.mode = "classification"
        self.verbose = True
        self.verbose_val = True
        self.smoothing = 0.1
        # optimizer
        self.patience = 5
        self.factor = 0.5
cfg = Config()

In [ ]:
backbone = timm.create_model(cfg.model_name, pretrained=False)
backbone.head = nn.Identity()
backbone.head_dist = nn.Identity()
model = PetNet(backbone, dim=768, num_classes=cfg.num_classes, output=cfg.mode, one_hot=cfg.one_hot)

model = model.to(cfg.device)
model.to_device(cfg.device)

In [ ]:
test_dataset = PetDataset(data=df_test, one_hot=cfg.one_hot, resize=cfg.resize)#PetDataset(data=new_images,  resize=224)
test_dataloader = DataLoader(test_dataset, num_workers=2, batch_size=cfg.batch_size, shuffle=False)
preds_avg = np.zeros((len(df_test),))
for i in range(cfg.kfold):
    ckpt_name = f"../input/petfinder-cait-m36-384/cait_m36_384-best-petfinders-classification-fold{i}.pth"

    model.load_state_dict(torch.load(ckpt_name, map_location="cpu"))
    #model = model.to(cfg.device)
    #model.to_device(cfg.device)
    preds = inference_fn(model, test_dataloader, cfg)
    #preds_max = np.argmax(preds, axis=1)
    preds_mean = (preds * np.array(range(cfg.num_classes)).reshape(1,-1)).sum(1)
    preds_avg += preds_mean

preds_cait_384 = preds_avg / cfg.kfold

In [ ]:
class Config(object):
    def __init__(self):
        self.epochs = 100
        self.lr = 1e-4
        self.wd = 1e-5
        self.accumulation = 1
        self.batch_size=16
        self.use_apex = True
        self.use_apex_val = False
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.num_classes = 101
        self.model_name = "cait_m48_448" #"cait_m36_384"#'swin_large_patch4_window12_384_in22k'
        self.kfold=5
        self.resize = 448
        self.one_hot= True
        self.mode = "classification"
        self.verbose = True
        self.verbose_val = True
        self.smoothing = 0.1
        # optimizer
        self.patience = 5
        self.factor = 0.5
cfg = Config()

In [ ]:
backbone = timm.create_model(cfg.model_name, pretrained=False)
backbone.head = nn.Identity()
backbone.head_dist = nn.Identity()
model = PetNet(backbone, dim=768, num_classes=cfg.num_classes, output=cfg.mode, one_hot=cfg.one_hot)

model = model.to(cfg.device)
model.to_device(cfg.device)

In [ ]:
test_dataset = PetDataset(data=df_test, one_hot=cfg.one_hot, resize=cfg.resize)#PetDataset(data=new_images,  resize=224)
test_dataloader = DataLoader(test_dataset, num_workers=2, batch_size=cfg.batch_size, shuffle=False)
preds_avg = np.zeros((len(df_test),))
for i in range(cfg.kfold):
    ckpt_name = f"../input/petfindertransformers/cait_m48_448-adam-aug-mixup-one-hot-newfolds/cait_m48_448-best-petfinders-classification-fold{i}.pth"

    model.load_state_dict(torch.load(ckpt_name, map_location="cpu"))
    #model = model.to(cfg.device)
    #model.to_device(cfg.device)
    preds = inference_fn(model, test_dataloader, cfg)
    #preds_max = np.argmax(preds, axis=1)
    preds_mean = (preds * np.array(range(cfg.num_classes)).reshape(1,-1)).sum(1)
    preds_avg += preds_mean

preds_cait_448 = preds_avg / cfg.kfold

In [ ]:
import gc

model = model.to("cpu")
del model

torch.cuda.empty_cache()

In [ ]:
# Tony

In [ ]:
import sys
sys.path.append('/kaggle/input/vit-keras-dataset')
!pip install -q /kaggle/input/validators0182-dataset/validators-0.18.2-py3-none-any.whl

In [ ]:
import sys
sys.path.append('../input/swintransformertf')
from swintransformer import SwinTransformer

In [ ]:
import os
import re
import cv2
import numpy as np
import pandas as pd
import joblib
from typing import Optional, Tuple
import warnings

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
# import efficientnet.tfkeras as efnB
from vit_keras import vit, utils, visualize, layers
from scipy.signal import get_window

from tensorflow.keras.losses import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

import tensorflow_hub as hub

from tqdm.notebook import tqdm

print(tf.__version__)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
strategy = tf.distribute.get_strategy()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Model Params
KFOLDS = 5
FOLD = 0

BATCH_SIZE = 16
AUG = False

In [ ]:
BASE_PATH = '/kaggle/input/petfinder-pawpularity-score'

df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df['image_path'] = BASE_PATH + '/train/' + df.Id + '.jpg'

test_df  = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['image_path'] = BASE_PATH + '/test/' + test_df.Id + '.jpg'

In [ ]:
"""
test_df2 = pd.DataFrame()

for i in range(len(test_df)):
    a = test_df.loc[i]
    d = pd.DataFrame(a).T
    test_df2 = test_df2.append([d]*850)  #每行复制24倍
test_df = test_df2
"""

In [ ]:
def parse_func(filename, image_size):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.keras.preprocessing.image.smart_resize(img, [image_size, image_size], interpolation='bilinear')

    img = tf.cast(img, tf.float32)
    img = tf.keras.applications.imagenet_utils.preprocess_input(img, mode="torch")

    img = tf.reshape(img, [image_size, image_size, 3])
    return img


def get_dataset(files, image_size, batch_size=BATCH_SIZE, aug=False):

    # ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.cache()

    ds = ds.map(lambda x: parse_func(x, image_size), num_parallel_calls=AUTO)

    """
    if aug:
        ds = ds.map(augment, num_parallel_calls=AUTO)
    """

    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
def build_model():
    inp = tf.keras.layers.Input(shape=[384, 384, 3])
    base = SwinTransformer('swin_large_384', include_top=False, pretrained=False, use_tpu=False)

    features = base(inp)
    x = tf.keras.layers.Dropout(0.3)(features)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp, outputs=[x, features])

    opt = tf.optimizers.Adam(learning_rate=1e-4)

    model.compile(opt, loss=tf.keras.losses.BinaryCrossentropy())

    return model

In [ ]:
import os
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt-174358-tfio', '/kaggle/input/petfinder-baseline-swt-174358-svr-all'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    # svr_paths = sorted(glob(os.path.join(svr_dir, 'svr-fold*.h5')))
    svr_default_paths = [os.path.join(svr_dir, f'svr-fold{i}.h5') for i in range(5)]
    svr_2_paths = [os.path.join(svr_dir, f'svr-fold-2-{i}.h5') for i in range(5)]
    svr_20_paths = [os.path.join(svr_dir, f'svr-fold-20-{i}.h5') for i in range(5)]

    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_default_paths, svr_2_paths, svr_20_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
preds_svr_2 = []
preds_svr_20 = []
for dim, model_paths, svr_default_paths, svr_2_paths, svr_20_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_default_path, svr_2_path, svr_20_path in zip(model_paths, svr_default_paths, svr_2_paths, svr_20_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        # svr default
        svr = joblib.load(svr_default_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)
        # svr 2
        svr = joblib.load(svr_2_path)
        pred_svr_2 = np.expand_dims(svr.predict(features), 1)
        # svr 20
        svr = joblib.load(svr_20_path)
        pred_svr_20 = np.expand_dims(svr.predict(features), 1)


        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]
        pred_svr_2 = pred_svr_2[:len(test_paths), :]
        pred_svr_20 = pred_svr_20[:len(test_paths), :]


        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        preds_svr_2.append(pred_svr_2) # denormalizing from [0-1] to [0-100]
        preds_svr_20.append(pred_svr_20) # denormalizing from [0-1] to [0-100]
        print()

preds_384 = np.mean(preds, axis=0)
preds_384_svr = np.mean(preds_svr, axis=0)
preds_384_svr_2 = np.mean(preds_svr_2, axis=0)
preds_384_svr_20 = np.mean(preds_svr_20, axis=0)

In [ ]:
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt-173328-soft', '/kaggle/input/petfinder-baseline-swt-173328-svr-all'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    # svr_paths = sorted(glob(os.path.join(svr_dir, 'svr-fold*.h5')))
    svr_default_paths = [os.path.join(svr_dir, f'svr-fold{i}.h5') for i in range(5)]
    svr_2_paths = [os.path.join(svr_dir, f'svr-fold-2-{i}.h5') for i in range(5)]
    svr_20_paths = [os.path.join(svr_dir, f'svr-fold-20-{i}.h5') for i in range(5)]

    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_default_paths, svr_2_paths, svr_20_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
preds_svr_2 = []
preds_svr_20 = []
for dim, model_paths, svr_default_paths, svr_2_paths, svr_20_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_default_path, svr_2_path, svr_20_path in zip(model_paths, svr_default_paths, svr_2_paths, svr_20_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        # svr default
        svr = joblib.load(svr_default_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)
        # svr 2
        svr = joblib.load(svr_2_path)
        pred_svr_2 = np.expand_dims(svr.predict(features), 1)
        # svr 20
        svr = joblib.load(svr_20_path)
        pred_svr_20 = np.expand_dims(svr.predict(features), 1)


        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]
        pred_svr_2 = pred_svr_2[:len(test_paths), :]
        pred_svr_20 = pred_svr_20[:len(test_paths), :]


        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        preds_svr_2.append(pred_svr_2) # denormalizing from [0-1] to [0-100]
        preds_svr_20.append(pred_svr_20) # denormalizing from [0-1] to [0-100]
        print()

preds_384_soft = np.mean(preds, axis=0)
preds_384_soft_svr = np.mean(preds_svr, axis=0)
preds_384_soft_svr_2 = np.mean(preds_svr_2, axis=0)
preds_384_soft_svr_20 = np.mean(preds_svr_20, axis=0)

In [ ]:
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt1k-174402-tfio', '/kaggle/input/petfinder-baseline-swt-174402-svr-all'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    # svr_paths = sorted(glob(os.path.join(svr_dir, 'svr-fold*.h5')))
    svr_default_paths = [os.path.join(svr_dir, f'svr-fold{i}.h5') for i in range(5)]
    svr_2_paths = [os.path.join(svr_dir, f'svr-fold-2-{i}.h5') for i in range(5)]
    svr_20_paths = [os.path.join(svr_dir, f'svr-fold-20-{i}.h5') for i in range(5)]

    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_default_paths, svr_2_paths, svr_20_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
preds_svr_2 = []
preds_svr_20 = []
for dim, model_paths, svr_default_paths, svr_2_paths, svr_20_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_default_path, svr_2_path, svr_20_path in zip(model_paths, svr_default_paths, svr_2_paths, svr_20_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        # svr default
        svr = joblib.load(svr_default_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)
        # svr 2
        svr = joblib.load(svr_2_path)
        pred_svr_2 = np.expand_dims(svr.predict(features), 1)
        # svr 20
        svr = joblib.load(svr_20_path)
        pred_svr_20 = np.expand_dims(svr.predict(features), 1)


        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]
        pred_svr_2 = pred_svr_2[:len(test_paths), :]
        pred_svr_20 = pred_svr_20[:len(test_paths), :]


        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        preds_svr_2.append(pred_svr_2) # denormalizing from [0-1] to [0-100]
        preds_svr_20.append(pred_svr_20) # denormalizing from [0-1] to [0-100]
        print()

preds_384_1k = np.mean(preds, axis=0)
preds_384_1k_svr = np.mean(preds_svr, axis=0)
preds_384_1k_svr_2 = np.mean(preds_svr_2, axis=0)
preds_384_1k_svr_20 = np.mean(preds_svr_20, axis=0)

In [ ]:
"""
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt1k-173756-soft', '/kaggle/input/petfinder-baseline-swt1k-173756-svr2'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    svr_paths = sorted(glob(os.path.join(svr_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
for dim, model_paths, svr_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_path in zip(model_paths, svr_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        svr = joblib.load(svr_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)

        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]

        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        print()

preds_384_1k_soft = np.mean(preds, axis=0)
preds_384_1k_soft_svr = np.mean(preds_svr, axis=0)
"""

In [ ]:
"""
def build_model():
    inp = tf.keras.layers.Input(shape=[384, 384, 3])
    base = SwinTransformer('swin_base_384', include_top=False, pretrained=False, use_tpu=False)

    features = base(inp)
    x = tf.keras.layers.Dropout(0.3)(features)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp, outputs=[x, features])

    opt = tf.optimizers.Adam(learning_rate=1e-4)

    model.compile(opt, loss=tf.keras.losses.BinaryCrossentropy())

    return model
"""

In [ ]:
"""
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt-base384-175904-tfio', '/kaggle/input/petfinder-baseline-swt-base384-175904-svr2'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    svr_paths = sorted(glob(os.path.join(svr_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
for dim, model_paths, svr_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_path in zip(model_paths, svr_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        svr = joblib.load(svr_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)

        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]

        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        print()

preds_384_base = np.mean(preds, axis=0)
preds_384_base_svr = np.mean(preds_svr, axis=0)
"""

In [ ]:
"""
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-swt-base384-175108-soft', '/kaggle/input/petfinder-baseline-swt-base384-175108-svr2'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    svr_paths = sorted(glob(os.path.join(svr_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
for dim, model_paths, svr_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_path in zip(model_paths, svr_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        svr = joblib.load(svr_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)

        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]

        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        print()

preds_384_base_soft = np.mean(preds, axis=0)
preds_384_base_soft_svr = np.mean(preds_svr, axis=0)
"""

In [ ]:
def build_model():
    inp = tf.keras.layers.Input(shape=[224, 224, 3])
    base = SwinTransformer('swin_large_224', include_top=False, pretrained=False, use_tpu=False)
    features = base(inp)
    x = tf.keras.layers.Dropout(0.3)(features)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp, outputs=[x, features])

    opt = tf.optimizers.Adam(learning_rate=1e-4)

    model.compile(opt, loss=tf.keras.losses.BinaryCrossentropy())

    return model

In [ ]:
from glob import glob

BASE_DIRS = [
    (224, '/kaggle/input/petfinder-baseline-swt224-175702-tfio', '/kaggle/input/petfinder-baseline-swt224-175702-svr-all'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    # svr_paths = sorted(glob(os.path.join(svr_dir, 'svr-fold*.h5')))
    svr_default_paths = [os.path.join(svr_dir, f'svr-fold{i}.h5') for i in range(5)]
    svr_2_paths = [os.path.join(svr_dir, f'svr-fold-2-{i}.h5') for i in range(5)]
    svr_20_paths = [os.path.join(svr_dir, f'svr-fold-20-{i}.h5') for i in range(5)]

    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_default_paths, svr_2_paths, svr_20_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
preds_svr_2 = []
preds_svr_20 = []
for dim, model_paths, svr_default_paths, svr_2_paths, svr_20_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_default_path, svr_2_path, svr_20_path in zip(model_paths, svr_default_paths, svr_2_paths, svr_20_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        # svr default
        svr = joblib.load(svr_default_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)
        # svr 2
        svr = joblib.load(svr_2_path)
        pred_svr_2 = np.expand_dims(svr.predict(features), 1)
        # svr 20
        svr = joblib.load(svr_20_path)
        pred_svr_20 = np.expand_dims(svr.predict(features), 1)


        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]
        pred_svr_2 = pred_svr_2[:len(test_paths), :]
        pred_svr_20 = pred_svr_20[:len(test_paths), :]


        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        preds_svr_2.append(pred_svr_2) # denormalizing from [0-1] to [0-100]
        preds_svr_20.append(pred_svr_20) # denormalizing from [0-1] to [0-100]
        print()

preds_224 = np.mean(preds, axis=0)
preds_224_svr = np.mean(preds_svr, axis=0)
preds_224_svr_2 = np.mean(preds_svr_2, axis=0)
preds_224_svr_20 = np.mean(preds_svr_20, axis=0)

In [ ]:
"""
from glob import glob

BASE_DIRS = [
    (224, '/kaggle/input/petfinder-baseline-swt224-174820-soft', '/kaggle/input/petfinder-baseline-swt224-174820-svr2'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    svr_paths = sorted(glob(os.path.join(svr_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
for dim, model_paths, svr_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_path in zip(model_paths, svr_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        svr = joblib.load(svr_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)

        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]

        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        print()

preds_224_soft = np.mean(preds, axis=0)
preds_224_soft_svr = np.mean(preds_svr, axis=0)
"""

In [ ]:
def parse_func(filename, image_size):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.keras.preprocessing.image.smart_resize(img, [image_size, image_size], interpolation='bilinear')

    img = tf.cast(img, tf.float32)
    img = vit.preprocess_inputs(img)

    img = tf.reshape(img, [image_size, image_size, 3])
    return img


def get_dataset(files, image_size, batch_size=BATCH_SIZE, aug=False):

    # ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.cache()

    ds = ds.map(lambda x: parse_func(x, image_size), num_parallel_calls=AUTO)

    """
    if aug:
        ds = ds.map(augment, num_parallel_calls=AUTO)
    """

    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTO)
    return ds

In [ ]:
def build_model():
    inp = tf.keras.layers.Input(shape=[384, 384, 3])
    base = getattr(vit, 'vit_l16')(image_size=384, include_top=False, pretrained_top=False, pretrained=False, weights='imagenet21k+imagenet2012')
    features = base(inp)
    x = tf.keras.layers.Dropout(0.3)(features)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inp, outputs=[x, features])

    opt = tf.optimizers.Adam(learning_rate=1e-4)

    model.compile(opt, loss=tf.keras.losses.BinaryCrossentropy())

    return model

In [ ]:
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-vit-176180-tfio', '/kaggle/input/petfinder-baseline-swt-176180-svr-all'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    # svr_paths = sorted(glob(os.path.join(svr_dir, 'svr-fold*.h5')))
    svr_default_paths = [os.path.join(svr_dir, f'svr-fold{i}.h5') for i in range(5)]
    svr_2_paths = [os.path.join(svr_dir, f'svr-fold-2-{i}.h5') for i in range(5)]
    svr_20_paths = [os.path.join(svr_dir, f'svr-fold-20-{i}.h5') for i in range(5)]

    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_default_paths, svr_2_paths, svr_20_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
preds_svr_2 = []
preds_svr_20 = []
for dim, model_paths, svr_default_paths, svr_2_paths, svr_20_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_default_path, svr_2_path, svr_20_path in zip(model_paths, svr_default_paths, svr_2_paths, svr_20_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        # svr default
        svr = joblib.load(svr_default_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)
        # svr 2
        svr = joblib.load(svr_2_path)
        pred_svr_2 = np.expand_dims(svr.predict(features), 1)
        # svr 20
        svr = joblib.load(svr_20_path)
        pred_svr_20 = np.expand_dims(svr.predict(features), 1)


        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]
        pred_svr_2 = pred_svr_2[:len(test_paths), :]
        pred_svr_20 = pred_svr_20[:len(test_paths), :]


        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        preds_svr_2.append(pred_svr_2) # denormalizing from [0-1] to [0-100]
        preds_svr_20.append(pred_svr_20) # denormalizing from [0-1] to [0-100]
        print()

preds_vit_384 = np.mean(preds, axis=0)
preds_vit_384_svr = np.mean(preds_svr, axis=0)
preds_vit_384_svr_2 = np.mean(preds_svr_2, axis=0)
preds_vit_384_svr_20 = np.mean(preds_svr_20, axis=0)

In [ ]:
"""
from glob import glob

BASE_DIRS = [
    (384, '/kaggle/input/petfinder-baseline-vit-175046-soft', '/kaggle/input/petfinder-baseline-vit-175046-svr2'),
]

MODEL_CONFIGS = []
for dim, base_dir, svr_dir in  BASE_DIRS:
    paths = sorted(glob(os.path.join(base_dir, '*h5')))
    svr_paths = sorted(glob(os.path.join(svr_dir, '*h5')))
    if len(paths)==0:
        print('no model found for :',base_dir)

    MODEL_CONFIGS.append([dim, paths, svr_paths])

print('='*35)
print('### Inference')
print('='*35)
preds=[]
preds_svr = []
for dim, model_paths, svr_paths in tqdm(MODEL_CONFIGS):
    test_paths = test_df.image_path.tolist()
    dtest = get_dataset(test_paths, dim)

    for model_path, svr_path in zip(model_paths, svr_paths):
        print(f'Model: {model_path}')

        with strategy.scope():
            model = build_model()
            model.load_weights(model_path)
            # model = tf.keras.models.load_model(model_path, custom_objects={'KerasLayer': hub.KerasLayer})

        print('Predicting...');
        pred, features = model.predict(dtest, verbose=1)

        svr = joblib.load(svr_path)
        pred_svr = np.expand_dims(svr.predict(features), 1)

        pred = pred[:len(test_paths),:]
        pred_svr = pred_svr[:len(test_paths), :]

        preds.append(pred*100.0) # denormalizing from [0-1] to [0-100]
        preds_svr.append(pred_svr) # denormalizing from [0-1] to [0-100]
        print()

preds_vit_384_soft = np.mean(preds, axis=0)
preds_vit_384_soft_svr = np.mean(preds_svr, axis=0)
"""

In [ ]:
# test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

features = np.stack([
    np.squeeze(preds_384),
    np.squeeze(preds_384_svr),
    np.squeeze(preds_384_svr_2),
    np.squeeze(preds_384_svr_20),

    np.squeeze(preds_384_soft),
    np.squeeze(preds_384_soft_svr),
    np.squeeze(preds_384_soft_svr_2),
    np.squeeze(preds_384_soft_svr_20),

    np.squeeze(preds_384_1k),
    np.squeeze(preds_384_1k_svr),
    np.squeeze(preds_384_1k_svr_2),
    np.squeeze(preds_384_1k_svr_20),

    # np.squeeze(preds_384_1k_soft),
    # np.squeeze(preds_384_1k_soft_svr),

    np.squeeze(preds_224),
    np.squeeze(preds_224_svr),
    np.squeeze(preds_224_svr_2),
    np.squeeze(preds_224_svr_20),

    # np.squeeze(preds_224_soft),
    # np.squeeze(preds_224_soft_svr),

    # np.squeeze(preds_384_base),
    # np.squeeze(preds_384_base_svr),

    # np.squeeze(preds_384_base_soft),
    # np.squeeze(preds_384_base_soft_svr),

    np.squeeze(preds_vit_384),
    np.squeeze(preds_vit_384_svr),
    np.squeeze(preds_vit_384_svr_2),
    np.squeeze(preds_vit_384_svr_20),

    # np.squeeze(preds_vit_384_soft),
    # np.squeeze(preds_vit_384_soft_svr),

    np.squeeze(preds_cait_384),
    np.squeeze(preds_cait_448),

    ], axis=1)

features = np.concatenate([features, test_df[columns].values], axis=1)


In [ ]:
lr_head = joblib.load('../input/petfinder-final-lr-170275/final-lr-170275.h5')
preds = lr_head.predict(features)

In [ ]:
pred_df = pd.DataFrame({'Id':test_df.Id,
                        'Pawpularity':preds.reshape(-1)})
sub_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/sample_submission.csv')
del sub_df['Pawpularity']
sub_df = sub_df.merge(pred_df, on='Id', how='left')
sub_df.to_csv('submission.csv',index=False)
sub_df.head()